In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ls drive/MyDrive/product_group_prediction/ProductGroupClassification/

dataset_groups_before_analyse.csv        functions.py
dataset_groups_preprocessing_100000.csv  preprocessing.py
dataset_sub_features.csv                 __pycache__/


In [ ]:
pip install pymorphy2 parapply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 8.5 MB/s 


In [ ]:
print('Import libs...')

import time
time_start = time.time()

import os
import sys
import random
import re
import joblib
import json

import warnings
warnings.filterwarnings("ignore")

import numpy as np
np.random.seed(42)
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import use

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score,recall_score, classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

import pymorphy2
# from natasha import (
#     Segmenter,
#     MorphVocab,
    
#     NewsEmbedding,
#     NewsMorphTagger,
#     NewsSyntaxParser,
#     NewsNERTagger,
    
#     PER,
#     NamesExtractor,

#     Doc
# )


import parapply
import importlib
importlib.reload(parapply)
from parapply import parapply

from tqdm import tqdm
tqdm.pandas()

import itertools
from itertools import product

import seaborn as sns



# import functions
# import importlib
# importlib.reload(functions)
# from functions import *




print('Successfully!')
print('process time: ',round(time.time()-time_start,2),' s')

Import libs...
Successfully!
process time:  0.0  s


In [ ]:
from functools import lru_cache


def voc_filter(x):
    import re
    x_clear = re.sub(r'\w*\d\w*', '', x).strip()
    return x_clear


# @lru_cache(maxsize=None) 
def lemmatizer(x):
    print(x)
    
    x_norm = []
    import pymorphy2
    morph = pymorphy2.MorphAnalyzer()
    
    for i in x:
        x_norm.append((morph.parse(i)[0]).normal_form)
    return x_norm


In [ ]:
ROOT_DIR = '/content/drive/MyDrive/product_group_prediction/ProductGroupClassification'

dataset_sub = pd.read_csv(ROOT_DIR + '/dataset_groups_preprocessing_100000.csv',sep=';',dtype=object)

dataset_sub = dataset_sub.dropna()
print(dataset_sub.shape)
dataset_sub = dataset_sub.reset_index(drop=True)

(95464, 5)


In [ ]:
dataset_sub.head(10)

,artical,brend_code,desc_clear_norm,guid,group_code
0,DF2591,823,тормозной диск,427D643F-70AB-11E4-80D3-005056BCBDAA,131
1,AVX13X1600,8802473,ремень приводной зубчатый avx,E445F7AC-8A73-11E5-80ED-005056BC3ACC,487
2,9090506043,920,пружина возвратный,A115CAEE-74AF-11E4-A66C-AC220BCC8A13,355
3,CL0086,715,стойка стабилизатор,2A1477C6-3E1D-11EA-A6E2-00505681CFBE,322
4,RN1217,384,диск тормозной mitsubishi pajero pajero ...,535EFB51-CBEB-11E5-80F1-005056BC3ACC,131
5,DS2306R,13427775,привод передний правый,DE674F58-A30F-11E7-9B70-005056813EF5,401
6,8200366508,879,прокладка турбин катализатор kangoo ii megane ...,79E39FB3-9F83-11E4-80E0-005056BC3ACC,508
7,1608422680,11597601,молдинга перед бампер хром,1AB20D02-74AC-11E4-A66C-AC220BCC8A13,759
8,MA702,559,наконечник teknorot,19AC0A3F-70A9-11E4-80D3-005056BCBDAA,298
9,6PK1835,720,ремень поликлиновый,06C3D645-6FE3-11E4-80D3-005056BCBDAA,486


In [ ]:
# dataset_sub_features.to_csv(ROOT_DIR + '/dataset_sub_features.csv',sep=';',index=False)
dataset_sub_features = pd.read_csv(ROOT_DIR + '/dataset_sub_features.csv',sep=';')



enc = LabelEncoder()
dataset_sub['target'] = enc.fit_transform(dataset_sub['group_code'])
dataset_sub_stats = pd.DataFrame(dataset_sub['target'].value_counts()).reset_index()
dataset_sub_stats.head(10)

,index,target
0,356,4544
1,336,3826
2,291,3443
3,144,2695
4,146,2557
5,340,2361
6,148,2048
7,166,1980
8,319,1811
9,339,1752


In [ ]:
dataset_sub[dataset_sub['target']==166]

,artical,brend_code,desc_clear_norm,guid,group_code,target
111,0125ACA25UP,346351,тяга toyota зад подть,AEE89F61-6FFA-11E4-80D3-005056BCBDAA,1558,166
182,33326770980,693,реактивный тяга плата зд,1AD8EFA5-74AD-11E4-A66C-AC220BCC8A13,1558,166
263,VKDS323168B,897,рычаг нижний левый citroen berlingo peugeot p...,72B06328-EB39-11E9-A753-00505681CFBE,1558,166
270,31126788982,693,поперечный рычаг плата вх,022DC439-74AD-11E4-A66C-AC220BCC8A13,1558,166
279,CAH0101D,325,рычаг подвеска,B241B59D-911C-11E6-99FF-00505686CE01,1558,166
...,...,...,...,...,...,...
95253,DH319,559,рычаг передний подвеска нижний левый,F1943C46-70A8-11E4-80D3-005056BCBDAA,1558,166
95294,JTC1064,823,рычаг независимый подвеска колесо,F7FE1452-70AC-11E4-80D3-005056BCBDAA,1558,166
95336,DB62044,19541908,рычаг audi vw passat переть подть нижный з...,1046BDB7-A2FB-11E7-9B70-005056813EF5,1558,166
95378,RA31021,16189157,рычаг подвеска,67A9FF91-7E87-11EC-8F56-005056810133,1558,166


In [ ]:
dataset_sub_stats = dataset_sub_stats.rename(columns={'index':'target','target':'count'})
dataset_sub_stats.columns


Index(['target', 'count'], dtype='object')

In [ ]:
dataset_sub['target'].value_counts()

356    4544
336    3826
291    3443
144    2695
146    2557
       ... 
136       1
486       1
79        1
236       1
40        1
Name: target, Length: 861, dtype: int64

In [ ]:
dataset_sub['brend_code'].unique().shape

(1241,)

In [ ]:
import pandas as pd

def OhePreprocessing(dataset, target=True, train_bool=True, cat_dummies = None, train_cols_order = None):
    cols=list(dataset.columns)

    if target:
        dataset_ohe_form = dataset[['ISU', 'ST_YEAR', 'SEMESTER', 'DISC_ID', 'DEBT']]
    else:
        dataset_ohe_form = dataset[['ISU', 'ST_YEAR', 'SEMESTER', 'DISC_ID']]

    cols.remove('ISU')
    cols.remove('ST_YEAR')
    cols.remove('SEMESTER')
    cols.remove('DISC_ID')
    # cols.remove('TYPE_NAME')
    if 'DEBT' in cols:
        cols.remove('DEBT')

    for col in cols:
        print(col)
        if pd.api.types.is_object_dtype(dataset[col]):
            df = pd.get_dummies(dataset[col], prefix=str(col), prefix_sep="__",
                              columns=dataset[col])
        else:
            df = pd.DataFrame(dataset[col])
        dataset_ohe_form = pd.concat((dataset_ohe_form,df),axis=1)


    if train_bool:
        cat_dummies = [col for col in dataset_ohe_form
                   if "__" in col
                   and col.split("__")[0] in cols]
        train_cols_order = list(dataset_ohe_form.columns)

    else:
        for col in dataset_ohe_form.columns:
            if ("__" in col) and (col.split("__")[0] in cols) and col not in cat_dummies:
                print("Removing additional feature {}".format(col))
                dataset_ohe_form.drop(col, axis=1, inplace=True)

        for col in cat_dummies:
            if col not in dataset_ohe_form.columns:
                print("Adding missing feature {}".format(col))
                dataset_ohe_form[col] = 0

        if target:
            dataset_ohe_form = dataset_ohe_form[train_cols_order]
        else:
            train_cols_order.remove('DEBT')


    if train_bool:
        return dataset_ohe_form, cat_dummies, train_cols_order
    else:
        return dataset_ohe_form[train_cols_order]

In [ ]:
dataset_sub.dtypes

artical            object
brend_code         object
desc_clear_norm    object
guid               object
group_code         object
target              int64
dtype: object

In [ ]:
dataset_sub_brand_ohe = pd.get_dummies(dataset_sub['brend_code'],prefix='brand')
dataset_sub_brand_ohe.head(5)

,brand_100160146,brand_100666204,brand_101047934,brand_1017688,brand_1018672,brand_102,brand_104725686,brand_1050182462,brand_106,brand_10876829,...,brand_9751237,brand_9751583,brand_9752026,brand_981,brand_981056076,brand_9828778,brand_9859048,brand_986,brand_993,brand_994
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(dataset_sub.shape,dataset_sub_brand_ohe.shape,dataset_sub_features.shape)

(95464, 6) (95464, 1241) (95464, 1407)


In [ ]:
print(dataset_sub_brand_ohe.dtypes, dataset_sub_features.iloc[:,4:].dtypes)

brand_100160146    uint8
brand_100666204    uint8
brand_101047934    uint8
brand_1017688      uint8
brand_1018672      uint8
                   ...  
brand_9828778      uint8
brand_9859048      uint8
brand_986          uint8
brand_993          uint8
brand_994          uint8
Length: 1241, dtype: object group_code         int64
abs              float64
absorber         float64
accent           float64
accord           float64
                  ...   
эксцентрик       float64
эл               float64
электрический    float64
элемент          float64
эмблема          float64
Length: 1403, dtype: object


In [ ]:
dataset_train_test = pd.concat((pd.DataFrame(dataset_sub['target']), dataset_sub_brand_ohe, dataset_sub_features.iloc[:,4:]),axis=1)
dataset_train_test['target'] = dataset_train_test['target'].astype(int)
X_train, X_test, y_train, y_test = train_test_split(dataset_train_test.drop(['target'],axis=1),dataset_train_test['target'],test_size=0.3,random_state=42)

X_train.head(5)

,brand_100160146,brand_100666204,brand_101047934,brand_1017688,brand_1018672,brand_102,brand_104725686,brand_1050182462,brand_106,brand_10876829,...,шт уп,шт цена,щётка,щётка стеклоочиститель,щёткодержатель,эксцентрик,эл,электрический,элемент,эмблема
58371,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48004,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80965,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45844,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4427,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y_train.dtypes

dtype('int64')

In [ ]:
y_train.shape

(66824,)

In [ ]:
lr = LogisticRegression(C=1,random_state=42,n_jobs=-1,class_weight='balanced')
lr.fit(X_train, y_train)





LogisticRegression(C=1, class_weight='balanced', n_jobs=-1, random_state=42)

In [ ]:
y_test_val = pd.DataFrame(y_test,columns=['target'])
# y_test_val = y_test_val.drop(['predict'],axis=0)
y_test_val['predict'] = lr.predict(X_test)

y_train_val = pd.DataFrame(y_train,columns=['target'])
# y_train_val = y_train_val.drop(['predict'],axis=0)
y_train_val['predict'] = lr.predict(X_train)

report = classification_report(y_test_val['target'], y_test_val['predict'], output_dict=True)
test_report = pd.DataFrame(report).transpose()

report = classification_report(y_train_val['target'], y_train_val['predict'], output_dict=True)
train_report = pd.DataFrame(report).transpose()



In [ ]:
test_report.sort_values(by='f1-score',ascending=False)

,precision,recall,f1-score,support
595,1.0,1.0,1.0,1.0
111,1.0,1.0,1.0,1.0
608,1.0,1.0,1.0,1.0
184,1.0,1.0,1.0,3.0
153,1.0,1.0,1.0,1.0
...,...,...,...,...
118,0.0,0.0,0.0,0.0
451,0.0,0.0,0.0,3.0
450,0.0,0.0,0.0,0.0
119,0.0,0.0,0.0,1.0


In [ ]:
train_report.sort_values(by='f1-score',ascending=False)

,precision,recall,f1-score,support
251,1.0,1.0,1.0,1.0
20,1.0,1.0,1.0,1.0
177,1.0,1.0,1.0,1.0
839,1.0,1.0,1.0,1.0
157,1.0,1.0,1.0,1.0
...,...,...,...,...
712,0.0,0.0,0.0,9.0
713,0.0,0.0,0.0,2.0
716,0.0,0.0,0.0,3.0
717,0.0,0.0,0.0,35.0


In [ ]:
y_test_val.dtypes

target     object
predict     int64
dtype: object

In [ ]:
y_test_val['predict'].value_counts()

166    28632
356        8
Name: predict, dtype: int64

In [ ]:
y_test_val['target'].value_counts()

356    1427
336    1108
291    1001
144     779
146     770
       ... 
121       1
526       1
101       1
133       1
847       1
Name: target, Length: 756, dtype: int64

In [ ]:
test_report.to_excel(ROOT_DIR+ '/test_report.xlsx',index=False)
train_report.to_excel(ROOT_DIR+ '/train_report.xlsx',index=False)

In [ ]:
accuracy_test = accuracy_score(y_test_val['target'], y_test_val['predict'])
accuracy_train = accuracy_score(y_train_val['target'], y_train_val['predict'])
print('accuracy_test=',accuracy_test,'accuracy_train=',accuracy_train)

accuracy_test= 0.388372905027933 accuracy_train= 0.4030737459595355


In [ ]:
y_test_val['target'].value_counts()

356    1427
336    1108
291    1001
144     779
146     770
       ... 
121       1
526       1
101       1
133       1
847       1
Name: target, Length: 756, dtype: int64